File to train on the toy_df.csv (that came from the 001_Forbes Dataset)

In [7]:
import os
import ast
import torch
import random
import pandas as pd
from transformers import TapasConfig, TapasForQuestionAnswering, TapasTokenizer, AdamW

In [8]:
# load in tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# Function to convert the string representation to a list of tuples
def parse_answer_coords(coords_str):
    try:
        # Safely evaluate the string to a Python object
        coords = ast.literal_eval(coords_str)
        
        # Ensure the result is a list of tuples with integers
        if isinstance(coords, list) and all(
            isinstance(coord, (tuple, list)) and len(coord) == 2 and all(isinstance(x, int) for x in coord)
            for coord in coords
        ):
            return [tuple(coord) for coord in coords]  # Convert lists to tuples if needed
        else:
            raise ValueError(f"Invalid format for answer_coords: {coords_str}")
    except (ValueError, SyntaxError) as e:
        raise ValueError(f"Error parsing answer_coords: {coords_str}. Details: {e}")

In [10]:
# class to tokenizer dataset
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = self.process_answer_coords_column(data)
        self.tokenizer = tokenizer
        self.table_csv_path = 'data/'

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        table = pd.read_csv(self.table_csv_path + item.dataset).astype(
            str
        )  # be sure to make your table data text only
        encoding = self.tokenizer(
            table=table,
            queries=item.question,
            answer_coordinates=item.answer_coords,
            answer_text=item.sample_answer,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        # remove the batch dimension which the tokenizer adds by default
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        # add the float_answer which is also required (weak supervision for aggregation case)
        encoding["float_answer"] = torch.tensor(item.float_answer)
        return encoding

    def __len__(self):
        return len(self.data)
    
    # change answer_coords from strings
    def process_answer_coords_column(self, data, column_name="answer_coords"):
        if column_name not in data.columns:
            raise ValueError(f"Column '{column_name}' does not exist in the dataset.")

        def parse_answer_coords(coords_str):
            try:
                coords = ast.literal_eval(coords_str)
                if isinstance(coords, list) and all(
                    isinstance(coord, (tuple, list)) and len(coord) == 2 and all(isinstance(x, int) for x in coord)
                    for coord in coords
                ):
                    return [tuple(coord) for coord in coords]
                else:
                    raise ValueError(f"Invalid format for answer_coords: {coords_str}")
            except (ValueError, SyntaxError) as e:
                raise ValueError(f"Error parsing answer_coords: {coords_str}. Details: {e}")
        data[column_name] = data[column_name].apply(parse_answer_coords)
        return data

In [19]:
# load in toy_df, tokenize and place in dataloader
csv_path = 'data/toy_df.csv'
 
toy_df = pd.read_csv(csv_path)
datasets = list(set(toy_df['dataset']))
random.shuffle(datasets)
print(datasets)

dev_datasets = datasets[:2]
train_datasets = datasets[2:]
print(dev_datasets, train_datasets)
# Apply the function to the answer_coords column

# split into train and dev
toy_df_train = toy_df[toy_df['dataset'].isin(train_datasets)]
toy_df_dev = toy_df[toy_df['dataset'].isin(dev_datasets)]

# load train dataloader
train_dataset = TableDataset(toy_df_train, tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8)

# load dev dataloader
dev_dataset = TableDataset(toy_df_dev, tokenizer)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=8)

['004_Taxi.csv', '008_Tornados.csv', '007_Fifa.csv', '009_Central.csv', '011_SF.csv', '010_ECommerce.csv', '005_NYC.csv', '002_Titanic.csv', '001_Forbes.csv', '012_Heart.csv']
['004_Taxi.csv', '008_Tornados.csv'] ['007_Fifa.csv', '009_Central.csv', '011_SF.csv', '010_ECommerce.csv', '005_NYC.csv', '002_Titanic.csv', '001_Forbes.csv', '012_Heart.csv']


/var/folders/f1/_zcz3_s54kz_1zxxwgs414pw0000gn/T/ipykernel_66277/1796820079.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].apply(parse_answer_coords)


In [15]:
# load in WTQ model
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
)
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq", config=config)
optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
# finetune the WTQ model
model.train()
for epoch in range(10):
    print('On epoch:', epoch) # loop over the dataset multiple times
    for batch in train_dataloader:
        # get the inputs;
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        float_answer = batch["float_answer"]
        
        if isinstance(numeric_values, pd.Series):
            numeric_values = numeric_values.iloc[:] 
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=float_answer,
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
print('Done Training')

On epoch: 0


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

...

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

Done Training


In [ ]:
# try inference

# data is the dev_dataset
# inputs is the tokenized inputs (dev, but just queries and table)

for file in dev_datasets:
    print('dev on:', file)
    table = pd.read_csv(f'data/{file}').astype(str)
    dev_df = toy_df_dev[toy_df_dev['dataset'] == file]
    queries = list(dev_df['question'])
    inputs = tokenizer(
        table = table,
        queries = queries,
        padding="max_length",
        return_tensors="pt",
        truncation = True
    )

    outputs = model(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
    )

    # print results:
    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(table.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(table.iat[coordinate])
            answers.append(", ".join(cell_values))

    #display(table)
    #print("")

    i = 0
    real_answers = list(dev_df['sample_answer'])
    for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
        print(query)
        if predicted_agg == "NONE":
            print("Predicted answer: " + answer)
            print("Real answer: ", real_answers[i])
            i += 1
        else:
            print("Predicted answer: " + predicted_agg + " > " + answer)
            if predicted_agg == 'COUNT':
                count = answer.split(',')
                print('Counted: ', len(list(set(count))))
            print("Real answer: ", real_answers[i])
            i += 1
            
    print("")
    print("")

dev on: 004_Taxi.csv


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


In [ ]:
#TODO: calculate accuracy